In [1]:
# pip install torch

In [2]:
# !pip install transformers
!pip install datasets

In [3]:
# !pip install --upgrade transformers accelerate

In [4]:
import json, argparse, torch, sys, random, gc, os
import numpy as np
import re
import pandas as pd
import functools
from itertools import chain
from functools import partial
from pathlib import Path
import os
# Transformer
from transformers import (AutoTokenizer, Trainer, TrainingArguments,
                          AutoModelForTokenClassification, DataCollatorForTokenClassification,
                          DebertaV2Config, DebertaV2ForTokenClassification)
from datasets import Dataset, features
from typing import Iterable, Any, Callable
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML-Project/PII-DATA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train = pd.read_json("/content/drive/My Drive/ML-Project/PII-DATA/train.json")
test= pd.read_json('/content/drive/My Drive/ML-Project/PII-DATA/test.json')
traindf1 = pd.read_csv("/content/drive/My Drive/ML-Project/PII-DATA/pii_dataset.csv")

In [7]:
traindf1

,document,text,tokens,trailing_whitespace,labels,prompt,prompt_id,name,email,phone,job,address,username,url,hobby,len
0,1073d46f-2241-459b-ab01-851be8d26436,"My name is Aaliyah Popova, and I am a jeweler ...","['My', 'name', 'is', 'Aaliyah', 'Popova,', 'an...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Aaliyah Popova is a jeweler with 13 year...,1,Aaliyah Popova,aaliyah.popova4783@aol.edu,(95) 94215-7906,jeweler,97 Lincoln Street,NaN,NaN,Podcasting,363
1,5ec717a9-17ee-48cd-9d76-30ae256c9354,"My name is Konstantin Becker, and I'm a develo...","['My', 'name', 'is', 'Konstantin', 'Becker,', ...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Konstantin Becker is a developer with 2 ...,1,Konstantin Becker,konstantin.becker@gmail.com,0475 4429797,developer,826 Webster Street,NaN,NaN,Quilting,255
2,353da41e-7799-4071-ab20-d959b362612e,"As Mieko Mitsubishi, an account manager at a p...","['As', 'Mieko', 'Mitsubishi,', 'an', 'account'...","[True, True, True, True, True, True, True, Tru...","['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'O',...",\n Mieko Mitsubishi is a account manager. W...,3,Mieko Mitsubishi,mieko_mitsubishi@msn.org,+27 61 222 4762,account manager,1309 Southwest 71st Terrace,NaN,NaN,Metal detecting,259
3,9324ee01-7bdc-41b1-a7a5-01307f72c20d,"My name is Kazuo Sun, and I'm an air traffic c...","['My', 'name', 'is', 'Kazuo', 'Sun,', 'and', ""...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Kazuo Sun is a air traffic controller wi...,1,Kazuo Sun,kazuosun@hotmail.net,0304 2215930,air traffic controller,736 Sicard Street Southeast,NaN,NaN,Amateur radio,281
4,971fe266-2739-4f1b-979b-7f64e07d5a4a,"My name is Arina Sun, and I'm a dental hygieni...","['My', 'name', 'is', 'Arina', 'Sun,', 'and', ""...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Arina Sun is a dental hygienist. Write a...,3,Arina Sun,arina-sun@gmail.net,0412 1245924,dental hygienist,5701 North 67th Avenue,NaN,NaN,Related,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,a90b6b71-0e77-4089-a3a4-9a5b5c5aec78,"Hello, I'm Nicholas Moore, a man with a rich t...","['Hello,', ""I'm"", 'Nicholas', 'Moore,', 'a', '...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUDENT',...",\n Write a fictional semi-formal biography ...,0,Nicholas Moore,nicholas_moore8047@yahoo.gov,+91-63720 22261,videographer,35915 Patrick Mews Suite 978,n.moore,https://www.nmoore.org,Surfing,360
4430,1492ed0e-f162-424f-9c40-f3edde790ca1,"Hello, my name is Alexey Novikov and I'm a psy...","['Hello,', 'my', 'name', 'is', 'Alexey', 'Novi...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME...",\n Alexey Novikov is a psychologist. Write ...,3,Alexey Novikov,alexey.novikov@msn.gov,0264 828 4342,psychologist,161 Creek Road,NaN,NaN,Gardening,248
4431,57ef34c1-48db-4413-9573-774021e57f63,"My name is Ludmila Inoue, and I'm a person wit...","['My', 'name', 'is', 'Ludmila', 'Inoue,', 'and...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Write a fictional semi-formal biography ...,0,Ludmila Inoue,ludmila_inoue@outlook.net,(285) 815-7373,physician,706 Seagrove Road,NaN,NaN,Quilting,445
4432,a4486627-1c62-48b0-bf4f-53259ecc0a28,"Dr. Tu Garcia, a renowned dermatologist, embar...","['Dr.', 'Tu', 'Garcia,', 'a', 'renowned', 'der...","[True, True, True, True, True, True, True, Tru...","['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'O',...",\n Tu Garcia is a dermatologist. Write abou...,2,Tu Garcia,tugarcia@outlook.com,+91-14426 83047,dermatologist,1677 Anthony Run,t.garcia,http://blog.tu-garcia.edu,Skiing,320


In [8]:
train

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [9]:
import pandas as pd

# Assuming 'df' is your pandas DataFrame and 'column_name' is the name of the column you're interested in
max_value = train['document'].max()

print(max_value)


22687


In [10]:
import ast

# Check the type of the first element in the 'tokens' and 'labels' columns
print(type(train['tokens'].iloc[0]))
print(type(train['labels'].iloc[0]))

# If the types are strings and you need to convert them to lists
if isinstance(train['tokens'].iloc[0], str):
    train['tokens'] = train['tokens'].apply(ast.literal_eval)
if isinstance(train['labels'].iloc[0], str):
    train['labels'] = train['labels'].apply(ast.literal_eval)


<class 'list'>
<class 'list'>


In [11]:
import ast

# Check the type of the first element in the 'tokens' and 'labels' columns
print(type(traindf1['tokens'].iloc[0]))
print(type(traindf1['labels'].iloc[0]))

# If the types are strings and you need to convert them to lists
if isinstance(traindf1['tokens'].iloc[0], str):
    traindf1['tokens'] = traindf1['tokens'].apply(ast.literal_eval)
if isinstance(traindf1['labels'].iloc[0], str):
    traindf1['labels'] = traindf1['labels'].apply(ast.literal_eval)

print(type(traindf1['tokens'].iloc[0]))
print(type(traindf1['labels'].iloc[0]))


<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>


In [12]:

#building up train data and test data
train_data1 = pd.DataFrame({"document":traindf1["document"],"words":traindf1["tokens"],"labels":traindf1['labels']})

In [13]:
train.head()


,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [14]:

#building up train data and test data
train_data = pd.DataFrame({"document":train["document"],"words":train["tokens"],"labels":train['labels']})

In [15]:
train_data.head()


,document,words,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [16]:
train_data1.head()

,document,words,labels
0,1073d46f-2241-459b-ab01-851be8d26436,"[My, name, is, Aaliyah, Popova,, and, I, am, a...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
1,5ec717a9-17ee-48cd-9d76-30ae256c9354,"[My, name, is, Konstantin, Becker,, and, I'm, ...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
2,353da41e-7799-4071-ab20-d959b362612e,"[As, Mieko, Mitsubishi,, an, account, manager,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."
3,9324ee01-7bdc-41b1-a7a5-01307f72c20d,"[My, name, is, Kazuo, Sun,, and, I'm, an, air,...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4,971fe266-2739-4f1b-979b-7f64e07d5a4a,"[My, name, is, Arina, Sun,, and, I'm, a, denta...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."


In [17]:
import pandas as pd

# Assuming 'train_data' is the original DataFrame you're working with.

# Create a new DataFrame by repeating the doc_id for each token
expanded_doc_id = train_data.explode('words')['document'].reset_index(drop=True)

# Explode the words and labels columns to get a series of tokens and labels
tokens = train_data['words'].explode().reset_index(drop=True)
labels = train_data['labels'].explode().reset_index(drop=True)

# Now create the result DataFrame from the expanded series
result_df = pd.DataFrame({
    'document': expanded_doc_id,
    'token': tokens,
    'label': labels
})

result_df


,document,token,label
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
4992528,22687,process,O
4992529,22687,explained,O
4992530,22687,above,O
4992531,22687,.,O


In [18]:
# Remove rows where 'token' is whitespace, a newline character, or a dot
result_df = result_df[~result_df['token'].str.match(r'^\s*$|^\\n$|^\.$')]

# If there are any specific strings you consider useless, you can add them to the condition
# For example, to remove rows with tokens that are just 'n\n' you can do the following:
result_df = result_df[result_df['token'] != 'n\n']

# Reset the index after the removal
result_df = result_df.reset_index(drop=True)

result_df

,document,token,label
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
4434368,22687,of,O
4434369,22687,the,O
4434370,22687,process,O
4434371,22687,explained,O


In [19]:
result_df['label'].value_counts()

O                   4431638
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         16
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: label, dtype: int64

In [20]:
import pandas as pd

# Assuming the dataframe 'df' contains the data and the 'doc_id' column needs to be replaced with numbers starting from 1
train_data1['document'] = range(25000, len(train_data1) + 25000)

# Display the first few rows of the dataframe to confirm the change
train_data1

,document,words,labels
0,25000,"[My, name, is, Aaliyah, Popova,, and, I, am, a...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
1,25001,"[My, name, is, Konstantin, Becker,, and, I'm, ...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
2,25002,"[As, Mieko, Mitsubishi,, an, account, manager,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."
3,25003,"[My, name, is, Kazuo, Sun,, and, I'm, an, air,...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4,25004,"[My, name, is, Arina, Sun,, and, I'm, a, denta...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
...,...,...,...
4429,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
4430,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4431,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4432,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [21]:
import pandas as pd

# Assuming 'train_data' is the original DataFrame you're working with.

# Create a new DataFrame by repeating the doc_id for each token
expanded_doc_id = train_data1.explode('words')['document'].reset_index(drop=True)

# Explode the words and labels columns to get a series of tokens and labels
tokens = train_data1['words'].explode().reset_index(drop=True)
labels = train_data1['labels'].explode().reset_index(drop=True)

# Now create the result DataFrame from the expanded series
result_df1 = pd.DataFrame({
    'document': expanded_doc_id,
    'token': tokens,
    'label': labels
})

# Replace any commas and periods in the 'token' column
result_df1['token'] = result_df1['token'].str.replace('[,.]', '', regex=True)

result_df1

,document,token,label
0,25000,My,O
1,25000,name,O
2,25000,is,O
3,25000,Aaliyah,B-NAME_STUDENT
4,25000,Popova,I-NAME_STUDENT
...,...,...,...
1373340,29433,overall,O
1373341,29433,success,O
1373342,29433,of,O
1373343,29433,the,O


In [22]:
result_df

,document,token,label
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
4434368,22687,of,O
4434369,22687,the,O
4434370,22687,process,O
4434371,22687,explained,O


In [23]:
import pandas as pd

# Assuming result_df1 and result_df2 are your two dataframes that you want to join
result_df = pd.concat([result_df, result_df1], ignore_index=True)

result_df


,document,token,label
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
5807713,29433,overall,O
5807714,29433,success,O
5807715,29433,of,O
5807716,29433,the,O


In [24]:
# Strip whitespace from the labels in the DataFrame before mapping
result_df['label'] = result_df['label'].str.strip()

# Recreate the label map in case there are leading/trailing whitespaces in the labels
unique_labels = result_df['label'].unique()
label_map = {label: idx for idx, label in enumerate(unique_labels)}

# Try mapping again
result_df['label'] = result_df['label'].map(label_map)


In [25]:
label_map

{'O': 0,
 'B-NAME_STUDENT': 1,
 'I-NAME_STUDENT': 2,
 'B-URL_PERSONAL': 3,
 'B-EMAIL': 4,
 'B-ID_NUM': 5,
 'I-URL_PERSONAL': 6,
 'B-USERNAME': 7,
 'B-PHONE_NUM': 8,
 'I-PHONE_NUM': 9,
 'B-STREET_ADDRESS': 10,
 'I-STREET_ADDRESS': 11,
 'I-ID_NUM': 12}

In [26]:
result_df['label'].value_counts()

0     5765152
1       12469
11       8593
2        6763
4        3833
10       3545
9        3404
8        2425
3         730
7         724
5          78
6           1
12          1
Name: label, dtype: int64

In [27]:
# To save this DataFrame to a CSV file
result_df.to_csv('/content/drive/My Drive/ML-Project/PII-DATA/cleandata.csv',index=False)


In [28]:
from transformers import BertTokenizerFast, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Group the tokens by 'doc_id' and create a list of lists of tokens
grouped = result_df.groupby('document')['token'].apply(list)
tokens = grouped.tolist()

# Tokenize all tokens at once
encoding = tokenizer.batch_encode_plus(
    tokens,
    is_split_into_words=True,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Extract input_ids and attention_masks
input_ids = encoding['input_ids']
attention_masks = encoding['attention_mask']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [29]:
# Prepare the labels
grouped_labels = result_df.groupby('document')['label'].apply(list)

# Calculate the maximum length after accounting for special tokens
max_length = 128  # this is the padding length you used for tokenization
max_label_length = max_length - 2  # Subtracting the special [CLS] and [SEP] tokens

labels = []
for label_list in grouped_labels:
    # Start with a label for the [CLS] token, followed by the actual labels
    label_array = [-100] + label_list

    # Add a label for the [SEP] token
    label_array += [-100]

    # Add padding if necessary to reach the max_label_length
    label_array += [-100] * (max_label_length - len(label_list))

    # Ensure the label array does not exceed the max_length
    label_array = label_array[:max_length]

    # Append the label array to the list of labels
    labels.append(torch.tensor(label_array))

# Stack the label tensors to create a single tensor
label_ids = torch.stack(labels)




# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(set(result_df['label'])),
    output_attentions=False,
    output_hidden_states=False,
)

# Move the model to the device
model.to(device)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [30]:
# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)



# Create the TensorDataset from the encoded inputs and labels
dataset = TensorDataset(input_ids, attention_masks, label_ids)


# Split dataset into train and validation sets
train_size = int(0.8 * len(input_ids))
val_size = len(input_ids) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for train and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Number of epochs to train for
epochs = 4

# Total number of training steps is the number of batches multiplied by the number of epochs
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_loader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

# Save the trained model to the specified directory
model.save_pretrained('/content/drive/My Drive/ML-Project/PII-DATA/model')  # Update this path as needed


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 | Train Loss: 0.0623
Epoch 2/4 | Train Loss: 0.0225
Epoch 3/4 | Train Loss: 0.0179
Epoch 4/4 | Train Loss: 0.0145


In [31]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Tracking variables for predictions and true labels
val_predictions, val_true_labels = [], []

# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode

for batch in validation_loader:
    # Transfer batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits and labels to CPU
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Collect the predictions and true labels for each batch
    val_predictions.extend(np.argmax(logits, axis=2).flatten())
    val_true_labels.extend(label_ids.flatten())

# Convert the lists into numpy arrays for evaluation
val_predictions = np.array(val_predictions)
val_true_labels = np.array(val_true_labels)

# Use the true labels and predictions to calculate the classification metrics
print('Validation Accuracy: {:.2f}'.format(accuracy_score(val_true_labels, val_predictions)))

# Use classification_report from sklearn to evaluate the model
report = classification_report(val_true_labels, val_predictions, labels=np.unique(val_true_labels), zero_division=0)
print(report)


Validation Accuracy: 0.99
              precision    recall  f1-score   support

        -100       0.00      0.00      0.00      2624
           0       0.99      1.00      0.99    281972
           1       0.89      0.60      0.71      1396
           2       0.89      0.88      0.88       960
           3       0.00      0.00      0.00        18
           4       0.80      0.04      0.07       102
           5       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        35
           8       0.13      0.07      0.09        56
           9       0.38      0.39      0.38        69
          10       0.59      0.45      0.51       185
          11       0.66      0.73      0.69       445

    accuracy                           0.99    287872
   macro avg       0.44      0.35      0.36    287872
weighted avg       0.98      0.99      0.98    287872



In [32]:
test

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."
5,86,Cheese Startup - Learning Launch ​by Eladio Am...,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[True, True, True, True, True, True, True, Fal..."
6,93,Silvia Villalobos\n\nChallenge:\n\nThere is a ...,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[True, False, False, False, False, False, True..."
7,104,Storytelling The Path to Innovation\n\nDr Sak...,"[Storytelling, , The, Path, to, Innovation, \...","[True, False, True, True, True, False, False, ..."
8,112,Reflection – Learning Launch\n\nFrancisco Ferr...,"[Reflection, –, Learning, Launch, \n\n, Franci...","[True, True, True, False, False, True, False, ..."
9,123,Gandhi Institute of Technology and Management ...,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr..."


In [33]:
test_data = pd.DataFrame()
#building up train data and test data
test_data = pd.DataFrame({"document":test["document"],"words":test["tokens"]})

In [34]:
test_data

,document,words
0,7,"[Design, Thinking, for, innovation, reflexion,..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind..."
4,56,"[Assignment, :, , Visualization, , Reflecti..."
5,86,"[Cheese, Startup, -, Learning, Launch, ​by, El..."
6,93,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,..."
7,104,"[Storytelling, , The, Path, to, Innovation, \..."
8,112,"[Reflection, –, Learning, Launch, \n\n, Franci..."
9,123,"[Gandhi, Institute, of, Technology, and, Manag..."


In [35]:
import pandas as pd

# Assuming 'test_data' contains the original DataFrame you're working with.

# Explode the 'words' column to have a separate row for each token
# while retaining the associated 'document' column value
test_data_exploded = test_data.explode('words').reset_index(drop=True)

# Use groupby on 'document' and cumcount to create a unique id for each token within each document
test_data_exploded['token_id'] = test_data_exploded.groupby('document').cumcount()

# Now your test_data_exploded contains a new column 'token_id' with a serial number for each token
test_df = pd.DataFrame({
    'document': test_data_exploded['document'],
    'token_id': test_data_exploded['token_id'],
    'token': test_data_exploded['words']
})
test_df

,document,token_id,token
0,7,0,Design
1,7,1,Thinking
2,7,2,for
3,7,3,innovation
4,7,4,reflexion
...,...,...,...
8500,123,1689,(
8501,123,1690,https://www.melessa.uni-
8502,123,1691,muenchen.de/team/vorstandssprecher/schmidt/pub...
8503,123,1692,)


In [36]:
import pandas as pd

submission_df = pd.read_csv('/content/drive/My Drive/ML-Project/PII-DATA/sample_submission.csv')


In [37]:
submission_df

,row_id,document,token,label
0,0,7,9,B-NAME_STUDENT
1,1,7,10,I-NAME_STUDENT
2,2,7,482,B-NAME_STUDENT
3,3,7,483,I-NAME_STUDENT
4,4,7,741,B-NAME_STUDENT
5,5,7,742,I-NAME_STUDENT
6,6,10,0,B-NAME_STUDENT
7,7,10,1,I-NAME_STUDENT
8,8,10,464,B-NAME_STUDENT
9,9,10,465,I-NAME_STUDENT


In [38]:
test_df

,document,token_id,token
0,7,0,Design
1,7,1,Thinking
2,7,2,for
3,7,3,innovation
4,7,4,reflexion
...,...,...,...
8500,123,1689,(
8501,123,1690,https://www.melessa.uni-
8502,123,1691,muenchen.de/team/vorstandssprecher/schmidt/pub...
8503,123,1692,)


In [39]:
import pandas as pd

# Assuming you have two dataframes: submission_df and test_df
# submission_df has 'row_id', 'document', 'token', 'label' columns
# test_df has 'document', 'token_id', and 'token' columns

# First, set the index of test_df to ['document', 'token_id'] to facilitate the mapping
test_df = test_df.set_index(['document', 'token_id'])

# Now, for each row in submission_df, find the corresponding token in test_df using the document and token_id
submission_df['token'] = submission_df.apply(lambda row: test_df.loc[(row['document'], row['token']), 'token'], axis=1)

print(submission_df)


    row_id  document       token           label
0        0         7    Nathalie  B-NAME_STUDENT
1        1         7       Sylla  I-NAME_STUDENT
2        2         7    Nathalie  B-NAME_STUDENT
3        3         7       Sylla  I-NAME_STUDENT
4        4         7    Nathalie  B-NAME_STUDENT
5        5         7       Sylla  I-NAME_STUDENT
6        6        10       Diego  B-NAME_STUDENT
7        7        10     Estrada  I-NAME_STUDENT
8        8        10       Diego  B-NAME_STUDENT
9        9        10     Estrada  I-NAME_STUDENT
10      10        16    Gilberto  B-NAME_STUDENT
11      11        16      Gamboa  I-NAME_STUDENT
12      12        20       Sindy  B-NAME_STUDENT
13      13        20      Samaca  I-NAME_STUDENT
14      14        56      Nadine  B-NAME_STUDENT
15      15        56        Born  I-NAME_STUDENT
16      16        86      Eladio  B-NAME_STUDENT
17      17        86       Amaya  I-NAME_STUDENT
18      18        93      Silvia  B-NAME_STUDENT
19      19        93

In [40]:
label_map

{'O': 0,
 'B-NAME_STUDENT': 1,
 'I-NAME_STUDENT': 2,
 'B-URL_PERSONAL': 3,
 'B-EMAIL': 4,
 'B-ID_NUM': 5,
 'I-URL_PERSONAL': 6,
 'B-USERNAME': 7,
 'B-PHONE_NUM': 8,
 'I-PHONE_NUM': 9,
 'B-STREET_ADDRESS': 10,
 'I-STREET_ADDRESS': 11,
 'I-ID_NUM': 12}

In [47]:
test_df = []

In [49]:
test_df = submission_df

In [51]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizerFast, BertForTokenClassification
from tqdm import tqdm
import numpy as np

  # Replace with the path to your test data

# Initialize the tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('/content/drive/My Drive/ML-Project/PII-DATA/model')  # Replace with your model path
model.to('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Group the tokens by document and create a list of lists of tokens
grouped = test_df.groupby('document')['token'].apply(list)
tokens = grouped.tolist()

# Tokenize all tokens at once using the tokenizer
encoded_inputs = tokenizer(tokens, is_split_into_words=True, padding=True, truncation=True, return_tensors="pt")
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Create a DataLoader
test_dataset = TensorDataset(input_ids, attention_mask)
test_loader = DataLoader(test_dataset, batch_size=32)

# Predict
predictions = []
for batch in tqdm(test_loader, desc="Predicting"):
    input_ids, attention_mask = tuple(t.to('cuda' if torch.cuda.is_available() else 'cpu') for t in batch)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions.extend(torch.argmax(logits, -1).cpu().numpy())

# Flatten the predictions
predictions = [item for sublist in predictions for item in sublist]

# Convert predictions to labels
id2label = {i: label for i, label in enumerate(model.config.id2label)}
predicted_labels = [id2label[pred] for pred in predictions]

# Map predictions to tokens, considering only the first prediction for each set of tokens
# corresponding to the same document. This is necessary because BERT might split tokens into subtokens
predicted_labels_per_document = []
for doc_id in grouped.index:
    length_of_doc = len(grouped[doc_id])
    # Slice the list of predicted labels to match the length of the current document's tokens
    doc_labels = predicted_labels[:length_of_doc]
    predicted_labels = predicted_labels[length_of_doc:]
    predicted_labels_per_document.extend(doc_labels)

# Check length consistency (optional)
assert len(predicted_labels_per_document) == len(test_df)

# Add predictions to DataFrame
test_df['predicted_label'] = predicted_labels_per_document




Predicting: 100%|██████████| 1/1 [00:00<00:00, 24.21it/s]

   row_id  document     token           label  predicted_label
0       0         7  Nathalie  B-NAME_STUDENT                0
1       1         7     Sylla  I-NAME_STUDENT                1
2       2         7  Nathalie  B-NAME_STUDENT                2
3       3         7     Sylla  I-NAME_STUDENT                0
4       4         7  Nathalie  B-NAME_STUDENT                0


In [52]:
test_df

,row_id,document,token,label,predicted_label
0,0,7,Nathalie,B-NAME_STUDENT,0
1,1,7,Sylla,I-NAME_STUDENT,1
2,2,7,Nathalie,B-NAME_STUDENT,2
3,3,7,Sylla,I-NAME_STUDENT,0
4,4,7,Nathalie,B-NAME_STUDENT,0
5,5,7,Sylla,I-NAME_STUDENT,0
6,6,10,Diego,B-NAME_STUDENT,0
7,7,10,Estrada,I-NAME_STUDENT,0
8,8,10,Diego,B-NAME_STUDENT,0
9,9,10,Estrada,I-NAME_STUDENT,0


In [54]:
# Define the label_map from the screenshot
label_map = {
    'O': 0,
    'B-NAME_STUDENT': 1,
    'I-NAME_STUDENT': 2,
    'B-URL_PERSONAL': 3,
    'B-EMAIL': 4,
    'B-ID_NUM': 5,
    'I-ID_NUM': 5,
    'I-URL_PERSONAL': 6,
    'B-USERNAME': 7,
    'B-PHONE_NUM': 8,
    'I-PHONE_NUM': 9,
    'B-STREET_ADDRESS': 10,
    'I-STREET_ADDRESS': 11,
    'I-ID_NUM': 12
}

# Create a reverse map from label IDs to labels
id2label = {id: label for label, id in label_map.items()}

# Function to convert numeric labels to strings using the id2label map
def get_label_name(label_id):
    return id2label.get(label_id, "Unknown_Label")  # Provides "Unknown_Label" for any ID not found in the map

# Apply this function to each element in the 'predicted_label' column of test_df
test_df['predicted_label_name'] = test_df['predicted_label'].apply(get_label_name)

# Display the DataFrame with the new 'predicted_label_name' column
print(test_df[['token', 'label', 'predicted_label', 'predicted_label_name']])


         token           label  predicted_label predicted_label_name
0     Nathalie  B-NAME_STUDENT                0                    O
1        Sylla  I-NAME_STUDENT                1       B-NAME_STUDENT
2     Nathalie  B-NAME_STUDENT                2       I-NAME_STUDENT
3        Sylla  I-NAME_STUDENT                0                    O
4     Nathalie  B-NAME_STUDENT                0                    O
5        Sylla  I-NAME_STUDENT                0                    O
6        Diego  B-NAME_STUDENT                0                    O
7      Estrada  I-NAME_STUDENT                0                    O
8        Diego  B-NAME_STUDENT                0                    O
9      Estrada  I-NAME_STUDENT                0                    O
10    Gilberto  B-NAME_STUDENT                0                    O
11      Gamboa  I-NAME_STUDENT                0                    O
12       Sindy  B-NAME_STUDENT                0                    O
13      Samaca  I-NAME_STUDENT    